# RSF模型训练

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import NearestNeighbors

def evaluate_model(params, X_train, y_train):
    kf = KFold(n_splits=5, shuffle=True, random_state=params.get('random_state'))
    c_indices_train = []

    # 归一化整个训练集
    scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    # 事件状态和SMOTE重采样
    y_train_events = y_train['event'].astype(int)
    smote = SMOTE(random_state=params.get('random_state'))
    X_train_resampled, y_train_resampled_events = smote.fit_resample(X_train_scaled, y_train_events)

    # 使用最近邻方法找到与重采样后样本最接近的原始样本的时间数据
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(X_train_scaled)
    _, indices = nn.kneighbors(X_train_resampled)
    y_train_resampled_times = y_train['time'][indices.flatten()]

    # 重新组装 y_train_resampled
    y_train_resampled = np.array([(event, time) for event, time in zip(y_train_resampled_events, y_train_resampled_times)],
                                 dtype=[('event', bool), ('time', '<f8')])

    # K折训练集C指数
    for train_index, test_index in kf.split(X_train_resampled):
        X_train_kf, X_test_kf = X_train_resampled[train_index], X_train_resampled[test_index]
        y_train_kf, y_test_kf = y_train_resampled[train_index], y_train_resampled[test_index]

        model = RandomSurvivalForest(**params)
        model.fit(X_train_kf, y_train_kf)

        predictions_train_kf = model.predict(X_test_kf)
        result_train_kf = concordance_index_censored(y_test_kf['event'], y_test_kf['time'], predictions_train_kf)
        c_index_train_kf = result_train_kf[0]
        c_indices_train.append(c_index_train_kf)

    mean_c_index_train = np.mean(c_indices_train)
    return mean_c_index_train

def evaluate_validation(params, X_train, y_train, X_val, y_val):
    scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model = RandomSurvivalForest(**params)
    model.fit(X_train_scaled, y_train)
    predictions_val = model.predict(X_val_scaled)
    result_val = concordance_index_censored(y_val['event'], y_val['time'], predictions_val)
    c_index_val = result_val[0]

    return c_index_val

# 加载和准备数据
data = pd.read_csv('dataset.csv')
X = data.drop(['follow_Up', 'outcome'], axis=1)
y = np.array([(event, time) for event, time in zip(data['outcome'], data['follow_Up'])],
             dtype=[('event', bool), ('time', '<f8')])

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ["sqrt", "log2", None],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'max_depth': [None, 3, 5, 10]
}

best_c_index_train = 0
best_params = None

random_seed = 45
np.random.seed(random_seed)

# 分层抽样划分数据集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_seed, stratify=data['outcome'])

# 参数搜索
for n_estimators in param_grid['n_estimators']:
    for max_features in param_grid['max_features']:
        for min_samples_split in param_grid['min_samples_split']:
            for min_samples_leaf in param_grid['min_samples_leaf']:
                for max_depth in param_grid['max_depth']:
                    params = {
                        'n_estimators': n_estimators,
                        'max_features': max_features,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'max_depth': max_depth,
                        'random_state': random_seed
                    }

                    mean_c_index_train = evaluate_model(params, X_train, y_train)
                    mean_c_index_val = evaluate_validation(params, X_train, y_train, X_val, y_val)

                    if mean_c_index_train <= 1.05 * mean_c_index_val and mean_c_index_train > best_c_index_train:
                        best_c_index_train = mean_c_index_train
                        best_params = params

# 最佳参数训练和验证集评估
if best_params:
    scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    best_model = RandomSurvivalForest(**best_params)
    best_model.fit(X_train_scaled, y_train)  # 注意这里使用原始数据，没使用重采样后的数据

    predictions_val = best_model.predict(X_val_scaled)
    result_val = concordance_index_censored(y_val['event'], y_val['time'], predictions_val)
    best_c_index_val = result_val[0]

    print(f"Best training C-index: {best_c_index_train} with parameters: {best_params}")
    print(f"Corresponding validation C-index: {best_c_index_val}")

Best training C-index: 0.8698152753650124 with parameters: {'n_estimators': 200, 'max_features': 'sqrt', 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10, 'random_state': 45}
Corresponding validation C-index: 0.831081081081081


In [5]:
import joblib

# 保存模型'RSFmodel.pkl'
joblib.dump(best_model, 'RSFmodel.pkl')

print("Model saved successfully as 'RSFmodel.pkl'.")

Model saved successfully as 'RSFmodel.pkl'.


# C-index + 95%CI

In [2]:
from sksurv.metrics import concordance_index_censored
from sklearn.utils import resample
import numpy as np

# 计算C-index
def evaluate_c_index(model, X, y):
    # 模型预测
    predictions = model.predict(X)

    # 计算C-index
    c_index_result = concordance_index_censored(y['event'], y['time'], predictions)
    c_index = c_index_result[0]

    return c_index

# Bootstrap方法计算95%置信区间
def bootstrap_ci(data, num_samples=100, alpha=0.95):
    bootstrap_estimates = np.array([np.mean(resample(data)) for _ in range(num_samples)])
    lower = np.percentile(bootstrap_estimates, (1 - alpha) / 2 * 100)
    upper = np.percentile(bootstrap_estimates, (1 + alpha) / 2 * 100)
    return lower, upper

# 获取模型的C-index
c_index_train = evaluate_c_index(best_model, X_train_scaled, y_train)
c_index_val = evaluate_c_index(best_model, X_val_scaled, y_val)

# 使用Bootstrap计算95%置信区间
bootstrap_results_train = []
bootstrap_results_val = []

for i in range(100):
    X_train_resampled, y_train_resampled = resample(X_train_scaled, y_train, random_state=i)
    X_val_resampled, y_val_resampled = resample(X_val_scaled, y_val, random_state=i)

    c_index_train_b = evaluate_c_index(best_model, X_train_resampled, y_train_resampled)
    c_index_val_b = evaluate_c_index(best_model, X_val_resampled, y_val_resampled)

    bootstrap_results_train.append(c_index_train_b)
    bootstrap_results_val.append(c_index_val_b)

# 计算95%置信区间
ci_train = bootstrap_ci(bootstrap_results_train, num_samples=100)
ci_val = bootstrap_ci(bootstrap_results_val, num_samples=100)

print(f"Training C-index: {c_index_train} with 95% CI: {ci_train}")
print(f"Validation C-index: {c_index_val} with 95% CI: {ci_val}")

Training C-index: 0.9698574338085539 with 95% CI: (0.9686399039042838, 0.9710947957900722)
Validation C-index: 0.831081081081081 with 95% CI: (0.8214652787470016, 0.84933513396068)


# time-dependent AUC + 95%CI

In [3]:
from sksurv.metrics import cumulative_dynamic_auc
from sklearn.utils import resample
import numpy as np
import warnings

# 计算time-dependent AUROC的函数，并计算平均AUROC
def calculate_time_dependent_auroc(model, X_train, y_train, X_val, y_val, times):
    # 获取训练集和验证集的风险分数
    risk_scores_train = model.predict(X_train)
    risk_scores_val = model.predict(X_val)
    
    # 计算每个时间点的time-dependent AUROC
    train_aucs = []
    val_aucs = []
    for time in times:
        train_auc, _ = cumulative_dynamic_auc(y_train, y_train, risk_scores_train, [time])
        val_auc, _ = cumulative_dynamic_auc(y_train, y_val, risk_scores_val, [time])
        train_aucs.append(train_auc[0])
        val_aucs.append(val_auc[0])
    
    # 计算平均AUROC（忽略NaN值）
    mean_train_auc = np.nanmean(train_aucs)
    mean_val_auc = np.nanmean(val_aucs)
    
    return mean_train_auc, mean_val_auc

# 使用Bootstrap计算95%置信区间的函数
def bootstrap_ci(data, num_samples=100, alpha=0.95):
    data = np.array(data)
    bootstrap_estimates = np.array([np.mean(resample(data, random_state=i)) for i in range(num_samples)])
    lower = np.percentile(bootstrap_estimates, (1 - alpha) / 2 * 100)
    upper = np.percentile(bootstrap_estimates, (1 + alpha) / 2 * 100)
    return lower, upper

# 计算95%置信区间的time-dependent AUROC
def calculate_time_dependent_auroc_with_bootstrap(model, X_train, y_train, X_val, y_val, n_bootstrap=100):
    # 使用训练集时间的分位数作为评估点
    times = np.percentile(y_train['time'], np.arange(10, 91, 10))
    # 确保时间点在验证集的随访时间范围内
    times = times[(times >= y_val['time'].min()) & (times < y_val['time'].max())]

    bootstrap_train_aucs = []
    bootstrap_val_aucs = []

    for i in range(n_bootstrap):
        # Bootstrap重采样
        X_train_resampled, y_train_resampled = resample(X_train, y_train, random_state=i)
        X_val_resampled, y_val_resampled = resample(X_val, y_val, random_state=i)

        # 确保时间点在resampled数据集的时间范围内
        current_times = times[(times >= y_val_resampled['time'].min()) & (times < y_val_resampled['time'].max())]

        # 如果没有有效的时间点，则跳过
        if len(current_times) == 0:
            continue

        # 计算bootstrap样本的time-dependent AUROC
        mean_train_auc, mean_val_auc = calculate_time_dependent_auroc(model, X_train_resampled, y_train_resampled, X_val_resampled, y_val_resampled, current_times)

        bootstrap_train_aucs.append(mean_train_auc)
        bootstrap_val_aucs.append(mean_val_auc)

    # 计算95%置信区间
    ci_train = bootstrap_ci(bootstrap_train_aucs)
    ci_val = bootstrap_ci(bootstrap_val_aucs)

    # 打印结果
    print("Bootstrap time-dependent AUROC 95% CI:")
    print(f"Training AUROC: mean={np.mean(bootstrap_train_aucs)}, 95% CI={ci_train}")
    print(f"Validation AUROC: mean={np.mean(bootstrap_val_aucs)}, 95% CI={ci_val}")

# 使用最佳模型计算time-dependent AUROC的95%置信区间
calculate_time_dependent_auroc_with_bootstrap(best_model, X_train_scaled, y_train, X_val_scaled, y_val)

C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encountered in divide
  true_pos = cumsum_tp / cumsum_tp[-1]
C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encountered in divide
  true_pos = cumsum_tp / cumsum_tp[-1]
C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encountered in divide
  true_pos = cumsum_tp / cumsum_tp[-1]
C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encountered in divide
  true_pos = cumsum_tp / cumsum_tp[-1]
C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encountered in divide
  true_pos = cumsum_tp / cumsum_tp[-1]
C:\Users\30952\AppData\Roaming\Python\Python311\site-packages\sksurv\metrics.py:482: RuntimeWarning: invalid value encou

Bootstrap time-dependent AUROC 95% CI:
Training AUROC: mean=0.9846780872313218, 95% CI=(0.9837842887469644, 0.9856201539832544)
Validation AUROC: mean=0.8840966345558501, 95% CI=(0.8739377922449449, 0.8968830221015152)


In [4]:
import numpy as np
from sksurv.metrics import integrated_brier_score
from sklearn.utils import resample

# 定义时间点范围
times = np.arange(12, 84)

# 定义一个函数来计算引导法的 IBS 和置信区间
def bootstrap_ibs(model, X, y, times, n_bootstrap=100, random_state=45):
    rng = np.random.default_rng(random_state)
    ibs_scores = []

    for _ in range(n_bootstrap):
        # 有放回地从原始数据中采样
        indices = rng.choice(len(y), len(y), replace=True)
        X_resampled = X[indices]
        y_resampled = y[indices]

        # 预测生存函数
        survs_resampled = model.predict_survival_function(X_resampled)

        # 计算预测概率
        preds_resampled = np.asarray([[fn(t) for t in times] for fn in survs_resampled])

        # 计算 IBS
        ibs_resampled = integrated_brier_score(y_resampled, y_resampled, preds_resampled, times)
        ibs_scores.append(ibs_resampled)

    # 计算 IBS 的平均值和 95% 置信区间
    ibs_mean = np.mean(ibs_scores)
    ci_lower, ci_upper = np.percentile(ibs_scores, [2.5, 97.5])
    
    return ibs_mean, ci_lower, ci_upper

# 使用引导法计算训练集的 IBS 及其 95% 置信区间
ibs_train_mean, ci_train_lower, ci_train_upper = bootstrap_ibs(best_model, X_train_scaled, y_train, times)

# 使用引导法计算验证集的 IBS 及其 95% 置信区间
ibs_val_mean, ci_val_lower, ci_val_upper = bootstrap_ibs(best_model, X_val_scaled, y_val, times)

print(f"Brier score for training set: {ibs_train_mean:.4f} (95% CI: {ci_train_lower:.4f}, {ci_train_upper:.4f})")
print(f"Brier score for validation set: {ibs_val_mean:.4f} (95% CI: {ci_val_lower:.4f}, {ci_val_upper:.4f})")


Brier score for training set: 0.0309 (95% CI: 0.0223, 0.0419)
Brier score for validation set: 0.1072 (95% CI: 0.0538, 0.1674)
